In [2]:
using HorizonSideRobots

In [3]:
robot = Robot("untitled.sit", animate=true)

Robot(HorizonSideRobots.SituationDatas.SituationData((0x000000000000000b, 0x000000000000000c), 1.0, true, (7, 10), [290 36 … 380 440; 211 189 … -141 -209; … ; -101 -51 … 81 208; -13 379 … -70 -109], Set([(-467, 7), (-60, 7), (-431, 7), (-504, 7), (-109, 7), (-55, 7), (-143, 7), (-89, 7), (-304, 7), (-419, 7)  …  (-334, 7), (-398, 7), (-438, 7), (-271, 7), (-356, 7), (-461, 7), (-215, 7), (-111, 7), (-312, 7), (-454, 7)]), Set{HorizonSide}[Set() Set() … Set() Set(); Set() Set() … Set() Set(); … ; Set() Set() … Set() Set(); Set() Set() … Set() Set()], (Scene (780px, 715px):
  0 Plots
  1 Child Scene:
    └ Scene (780px, 715px), Axis (16 plots))), true)

In [4]:
#=
function moveToBorder(robot, side, mark)
    while !isborder(robot, side)
        if mark
            putmarker!(robot)
        end
        move!(robot, side)
    end
end
function findSide(side) #
    if side == Nord
        return Ost
    elseif side == West
        return Nord
    elseif side == Sud
        return West
    elseif side == Ost
        return Sud
    end
end
function moveAlongRectangle(robot, side)#двигается вдоль прямоугольника(внутреннего)
    border_side = findSide(side)
    putmarker!(robot)
    move!(robot, side)
    while isborder(robot, border_side)
        putmarker!(robot)
        if isborder(robot, side)
            return true #внутреняя перегородка прилегает к внешней
        end
        move!(robot, side)
    end

    return false
end

function perimiter(robot)
    internal_border = true #встретили ли мы внутреннюю стенку
    while !isborder(robot, West)
        move!(robot, West)
    end

    while isborder(robot, West)
        if isborder(robot,Sud)
            internal_border = false
            break
        end
        putmarker!(robot)
        move!(robot, Sud)
    end

    if internal_border
        moveAlongRectangle(robot, West)
        for side in [Nord, Ost, Sud, West]
            moveAlongRectangle(robot, side)
        end
    end
end

perimiter(robot)
=#
#двигает робота в нижний Lевый угол
#двигает робота в нижний девый угол
function gotostart!(robot)
    x = 0
    y = 0
    steps = 0
    while !isborder(robot, West)
        move!(robot, West)
        x = x + 1
        steps += 1
    end
    while !isborder(robot, Sud)
        move!(robot, Sud)
        y = y + 1
        steps += 1
    end
    return x, y, steps
end

#возвращает робота в исходное положение
function return_to_start!(robot, x, y)
    gotostart!(robot)
    for i in 1:y
        move!(robot, Nord)
    end 
    for i in 1:x
        move!(robot, Ost)
    end
end

along!(robot::Robot,side::HorizonSide)::Nothing =
    while !isborder(robot,side)
        move!(robot,side)
    end

#робот движется в сторону до <условие>
function along!(robot, side, num_steps)
    for a in 1:num_steps move!(robot, side) end
end

#меняет сторону на противоположенную
function inverse(side::HorizonSide)
    HorizonSide((Int(side) + 2) % 4)
end

#возвращает размер клетки
function get_cage_size!(robot)
    x = 0
    y = 0
    while !isborder(robot, Ost)
        move!(robot, Ost)
        x += 1
    end
    while !isborder(robot, Nord)
        move!(robot, Nord)
        y += 1
    end
    gotostart!(robot)
    return x, y
end

#меняет сторону против часовой
function reverse90(side)
    if side == Nord
        return West
    elseif side == West
        return Sud
    elseif side == Sud
        return Ost
    elseif side == Ost
        return Nord
    end
end

#меняет сторону по часовой
function reverse_90(side)
    if side == Nord
        return Ost
    elseif side == Nord
        return Sud
    elseif side == West
        return Ost
    elseif side == Ost
        return Sud
    end
end

#определяет в какую сторону нужно поворачивать роботу(по часовой/против)
function direction(direc, side)
    if direc == 1
        return reverse90(side)
    else
        return reverse_90(side)
    end
end

#проставляет маркеры в направлении
function putmarkers!(robot::Robot,side::HorizonSide)::Nothing
    while !isborder(robot,side)
        move!(robot,side)
        putmarker!(robot)
    end
end

function try_move!(robot, direct)::Bool
    if isborder(robot, direct)
        return false
    end
    move!(robot, direct)
    return true
end


try_move!(robot, direct) = (!isborder(robot, direct) && (move!(robot, direct); return true); false)

sides = [Nord, Ost, Sud, West]

function inverse(side)
    if side == Nord
        return Sud
    elseif side == West
        return Ost
    elseif side == Sud
        return Nord
    elseif side == Ost
        return West
    end
end


function putmarkers!(robot::Robot,side::HorizonSide)::Nothing
    while !isborder(robot,side)
        move!(robot,side)
        putmarker!(robot)
    end
end

function rotate_side(r, side)

end

function makedoubleframe!(robot::Robot)
    x, y = gotostart!(robot)
	
	while !isborder(robot,Sud) || !isborder(robot,Ost)
		if !isborder(robot,Ost)
			move!(robot,Ost)
		end

		if !isborder(robot,Sud)
			move!(robot,Sud)
		end
	end

    for side in (HorizonSide(i) for i in 0:3)
        putmarkers!(robot,side)
    end

	side = West

	while !isborder(robot,Nord)
		move!(robot,side)

		if isborder(robot,side)
			side = inverse(side)
			move!(robot,Nord)
		end
	end

	if side == Ost
        direc = 1
    else
        direc = 0
    end


	for i in 0:3
		while isborder(robot,direction(direc, side))
			putmarker!(robot)
			move!(robot,side)
		end
		
		putmarker!(robot)

		if i >= 3 break end
		
		side = direction(direc, side)
		move!(robot,side)
	end
    return_to_start!(robot, x, y)
end

makedoubleframe!(robot)
